In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA A100-SXM4-80GB


In [1]:
from langchain_community.vectorstores import FAISS
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 로더 생성

LLM_MODEL_PATH = "../ai_models/llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M/llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M.gguf"
EMBEDDING_MODEL_PATH = "../ai_models/intfloat/multilingual-e5-large-instruct"  # 또는 로컬 모델 경로
VECTORSTORE_PATH = "../data/vectorstore/faiss_index" 


path = "../data/csv/한국인터넷진흥원_개인정보포털 상황별 FAQ정보_20240731.csv"
loader = CSVLoader(file_path=path, encoding="utf-8")


In [5]:
csv_docs = loader.load()

# 임베딩 모델 생성
embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_PATH)

# 문서 텍스트 추출 (Document 객체라면 .page_content 사용)
texts = [doc.page_content for doc in csv_docs]

# FAISS 인덱스 생성
vectorstore = FAISS.from_texts(texts, embedding_model)

# 인덱스 저장
vectorstore.save_local("../data/vectorstore/faiss_index")


In [6]:
embedding = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_PATH,
    model_kwargs={"device": "cpu"},                    # GPU 사용 안함
    encode_kwargs={"normalize_embeddings": True}       # 임베딩 정규화 (권장)
)
vectorstore = FAISS.load_local(VECTORSTORE_PATH, embedding, allow_dangerous_deserialization=True)

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})  # top-3 문서 검색

In [8]:
retriever.get_relevant_documents("개인정보 유출이란?")  # 검색 쿼리

/tmp/ipykernel_3983422/2043201211.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("개인정보 유출이란?")  # 검색 쿼리


[Document(id='e75bbb95-17d5-4fa0-b228-d46a9206e5d7', metadata={}, page_content='\ufeff처리상황단계내용: 안전성확보조치\n적용분야내용: 시스템 분야\n개인정보유형내용: 일반정보\n코드제목: 홈페이지 방문객이 게시판에 스스로 본인의 개인정보를 게시했다면?\n주제내용: 정보주체가 자신의 개인정보를 인터넷 게시판에 게재하는 경우 조치 방법\n문제상황내용: 협회에서는 홈페이지를 운영하고 있으나 별도의 회원가입 없이 누구나 자유롭게 게시판을 이용할 수 있도록 자유게시판을 운영하고 있습니다. 그런데 일부 홈페이지 이용자가 게시물에 자신의 성명, 전화번호, 이메일주소 등의 개인정보를 공개하는 경우가 있습니다. 이런 경우 개인정보가 유출되는 것은 아닌지 우려가 되고, 개인정보가 포함된 게시글을 삭제해야 하는지 고민이 됩니다.\n질문: 개인이 본인의 개인정보가 포함된 게시물을 작성하여 게시판에 올린 경우 어떻게 처리해야 하나요?\n해결방법내용: 홈페이지 운영자는 이용자가 작성한 게시물에 개인정보가 포함되어 있는 경우 개인정보의 추가적인 노출 및 타인에 의한 도용 등 2차적 피해를 예방하기 위하여  게시판 안내문을 통해 게시글에 개인정보가 포함되어 있는지 여부를 작성자가 확인하도록 하고, 개인정보 유출로 인한 피해가 있음을 알려줄 수 있습니다. 또한 고객이 부주의하게 게재한 개인정보는 고객의 의사를 확인하여 일부 마스킹 처리하거나 개인정보 입력을 사전에 차단할 수 있도록 ‘필터링 시스템’을 적용할 수 있습니다. 이와 같이 홈페이지 운영자는 최대한 개인정보 유출 방지를 위한 안전성 확보조치를 취하는 것이 바람직합니다.\n결론내용: 홈페이지 이용자가 인터넷 게시판을 이용하면서 부주의로 자신의 개인정보를 게재하지 않도록 피해가능성 등에 대한 경고문을 사전에 안내하고, 개인정보 유출 방지를 위한 안전조치를 하여야 할 것입니다.\n유사사례내용: FAQ_0000000000000748\nfaq식별자: FAQ_000000000

In [10]:
# 검색 쿼리 입력
query = "개인정보"

# top-k 유사 문서 검색
docs = vectorstore.similarity_search(query, k=3)

# 결과 출력
for i, doc in enumerate(docs):
    print(f"결과 {i+1}:\n{doc.page_content}\n")


결과 1:
﻿처리상황단계내용: 수집·이용
적용분야내용: 공통분야
개인정보유형내용: 일반정보
코드제목: 법인이나 개인사업자 정보도 개인정보로 보호되어야 한다?
주제내용: 법인 및 개인사업자 정보가 개인정보보호법상 개인정보에 해당 여부
문제상황내용: 우리 회사는 업무 효율성을 높이기 위해 협력사 및 거래처에 대한 정보를 체계적으로 관리하려고 합니다. 회사에서 관리하고자 하는 정보는 법인 및 개인사업자의 사업자등록번호, 사업장 주소, 사업장 전화번호, 대표자 성명입니다.
질문: 개인정보보호법에 따르면 개인정보는 살아 있는 개인에 대한 정보라고 정의 되어 있는데, 이러한 정보들이 개인정보에 해당되나요?
해결방법내용: 개인정보보호법은 생존하고 있는 자연인에 대한 개인정보를 보호 대상으로 하고 있습니다. 자연인이 아닌 법인이나 사업체(개인사업자 포함)의 정보는 원칙적으로 개인정보보호법상 보호대상인 개인정보에 해당하지 않습니다.그러나 대표자의 성명이 법인이나 사업체의 정보로서가 아니라 자연인으로서의 개인정보로 처리되는 경우 또는 마케팅 목적으로 이용되는 경우에는 개인정보보호법에 따른 개인정보에 포함될 수 있습니다.
결론내용: 법인이나 사업체의 정보인 사업장주소, 사업장전화번호, 대표자성명은 자연인으로서의 개인정보에 해당하지 않습니다
유사사례내용: FAQ_0000000000000770
faq식별자: FAQ_0000000000000784
최종수정년월일시: 2013-10-07 10:45:47
조회수: 11934
임시: 수집·이용

결과 2:
﻿처리상황단계내용: 안전성확보조치
적용분야내용: 시스템 분야
개인정보유형내용: 일반정보
코드제목: 홈페이지 방문객이 게시판에 스스로 본인의 개인정보를 게시했다면?
주제내용: 정보주체가 자신의 개인정보를 인터넷 게시판에 게재하는 경우 조치 방법
문제상황내용: 협회에서는 홈페이지를 운영하고 있으나 별도의 회원가입 없이 누구나 자유롭게 게시판을 이용할 수 있도록 자유게시판을 운영하고 있습니다. 그런데 일부 홈페이지 이용자가 게시물에 자신의 성명,